# Google Quick, Draw!

For generation: https://github.com/magenta/magenta/tree/main/magenta/models/sketch_rnn#pre-trained-models

## Download Data

In [1]:
!mkdir data/
!wget --show-progress http://download.tensorflow.org/data/quickdraw_tutorial_dataset_v1.tar.gz

mkdir: data/: File exists
--2022-04-19 23:48:13--  http://download.tensorflow.org/data/quickdraw_tutorial_dataset_v1.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.116.128
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.116.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1065301781 (1016M) [application/gzip]
Saving to: ‘quickdraw_tutorial_dataset_v1.tar.gz’

quickdraw_tutorial_ 100%[===================>]   1016M  24.3MB/s    in 45s     

2022-04-19 23:48:58 (22.5 MB/s) - ‘quickdraw_tutorial_dataset_v1.tar.gz’ saved [1065301781/1065301781]



In [2]:
!tar -xf quickdraw_tutorial_dataset_v1.tar.gz -C data/
!rm quickdraw_tutorial_dataset_v1.tar.gz

## Load Data

In [1]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [9]:
raw_image_dataset = tf.data.TFRecordDataset('./data/training.tfrecord-00000-of-00010')

image_feature_description = {
    'class_index': tf.io.FixedLenFeature([], tf.int64),
    'ink': tf.io.VarLenFeature(tf.float32),
    'shape': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {ink: (None,), class_index: (), shape: ()}, types: {ink: tf.float32, class_index: tf.int64, shape: tf.int64}>

In [6]:
raw_dataset = tf.data.TFRecordDataset('./data/training.tfrecord-00000-of-00010')
for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "class_index"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "ink"
    value {
      float_list {
        value: 0.019685029983520508
        value: 0.05095541477203369
        value: 0.0
        value: 0.03149610757827759
        value: 0.03184711933135986
        value: 0.0
        value: 0.12204724550247192
        value: -0.012738823890686035
        value: 0.0
        value: 0.06692910194396973
        value: -0.05095541477203369
        value: 0.0
        value: -0.015748023986816406
        value: -0.13375797867774963
        value: 0.0
        value: -0.0551181435585022
        value: -0.17834395170211792
        value: 0.0
        value: -0.04724407196044922
        value: -0.04458598792552948
        value: 0.0
        value: -0.04330706596374512
        value: -0.006369426846504211
        value: 0.0
        value: -0.0590551495552063
        value: 0.05095541477203369
        value: 0.0
        value: -

## Train Model

In [8]:
!python train_model.py \
    --training_data=data/training.tfrecord-00001-of-00010 \
    --eval_data=data/eval.tfrecord-00001-of-00010 \
    --model_dir=../models \
    --classes_file=data/training.tfrecord.classes


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0420 00:05:38.941774 4570850816 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


W0420 00:05:39.424884 4570850816 module_wrapper.py:139] From train_model.py:42: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Using config: {'_model_dir': '../models', '

In [2]:
!python classify.py \
    --classes_file=data/training.tfrecord.classes \
    --model_dir=../models/ \
    --training_data=./data/training.tfrecord-?????-of-????? \
    --eval_data=./data/eval.tfrecord-?????-of-????? \
    --predict_for_data="[[[73,66,46,23,12,11,22,48,58,67,70,65],[11,6,2,10,23,33,48,56,54,41,22,10]],[[66,85,71],[9,3,26]],[[24,1,2,8],[6,1,10,19]],[[64,88,134,176,180,184,184,174,111,63,47],[34,29,28,35,39,58,91,94,86,71,62]],[[64,61,62],[74,83,102]],[[83,84,87],[78,102,107]],[[157,159,164],[96,108,116]],[[175,182],[91,115]],[[182,186,198,209,223,234,251,255],[51,36,29,30,38,39,20,8]],[[157,136,128,133,139],[35,47,57,35,29]],[[104,94,84,84,89],[40,52,70,30,26]],[[111,105,105,109,121],[30,59,68,72,34]],[[159,153,153],[41,54,65]]]" \
    --predict_temp_file=./predict_temp.tfrecord \
    --steps=120000


2022-04-23 20:15:49.079383: I Starting application
FLAGS Namespace(batch_norm=False, batch_size=8, cell_type='lstm', classes_file='data/training.tfrecord.classes', conv_len='[5, 5, 3]', dropout=0.3, eval_data='./data/eval.tfrecord-?????-of-?????', gradient_clipping_norm=9.0, learning_rate=0.0001, model_dir='../models/', num_conv='[48, 64, 96]', num_layers=3, num_nodes=128, predict_for_data='[[[73,66,46,23,12,11,22,48,58,67,70,65],[11,6,2,10,23,33,48,56,54,41,22,10]],[[66,85,71],[9,3,26]],[[24,1,2,8],[6,1,10,19]],[[64,88,134,176,180,184,184,174,111,63,47],[34,29,28,35,39,58,91,94,86,71,62]],[[64,61,62],[74,83,102]],[[83,84,87],[78,102,107]],[[157,159,164],[96,108,116]],[[175,182],[91,115]],[[182,186,198,209,223,234,251,255],[51,36,29,30,38,39,20,8]],[[157,136,128,133,139],[35,47,57,35,29]],[[104,94,84,84,89],[40,52,70,30,26]],[[111,105,105,109,121],[30,59,68,72,34]],[[159,153,153],[41,54,65]]]', predict_temp_file='./predict_temp.tfrecord', self_test=True, steps=120000, training_data='.